# Countermeasures for side-channel analysis


#### Learning goals
- Learn how to avoid side-channel leakage
- Learn about _Hiding_ and _Masking_

#### References
- E. Oswald S. Mangard and T. Popp, Power analysis attacks — revealing the secrets
of smartcards, Springer-Verlag, 2007

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo

from securec.capture import capture

In [ ]:
class EvolutionOutputMethod(lascar.ScoreProgressionOutputMethod):
    def _finalize(self):
        scores = np.array(list(self.scores.values())[0]).T
        fig = pgo.Figure()
        for guess, score in zip(self.engines[0]._guess_range, scores):
            fig.add_trace(pgo.Scatter(x=self.steps, y=score, name=str(guess)))
        fig.show()

In [ ]:
def sbox_output_selection_function(value, guess):
    return lascar.hamming(lascar.tools.aes.sbox[value["input"][0] ^ guess])

def cpa(data, selection_function):
    trace = lascar.TraceBatchContainer(data["trace"], data)
    engine = lascar.CpaEngine(
        "cpa",
        selection_function=selection_function,
        guess_range=range(256),
    )

    session = lascar.Session(
        trace,
        engine=engine,
        output_method=EvolutionOutputMethod(engine),
        output_steps=range(0, len(data), 100),
        progressbar=False,
    )
    session.run(batch_size="auto")

## 1. Hiding

### 1 Bit random

In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=500,
    number_of_samples=500,
    fromfile=os.path.abspath("sbox_lookup_hiding.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)] + [random.randint(0, 1)],
)

cpa(data, sbox_output_selection_function)


### 2 Bit random

In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=1000,
    number_of_samples=500,
    fromfile=os.path.abspath("sbox_lookup_hiding.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)] + [random.randint(0, 3)],
)

cpa(data, sbox_output_selection_function)


### 3 Bit random

In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=5000,
    number_of_samples=500,
    fromfile=os.path.abspath("sbox_lookup_hiding.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)] + [random.randint(0, 7)],
)

cpa(data, sbox_output_selection_function)


### 4 Bit random

In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=10000,
    number_of_samples=500,
    fromfile=os.path.abspath("sbox_lookup_hiding.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)] + [random.randint(0, 15)],
)

cpa(data, sbox_output_selection_function)


## 2. Masking

### Masking the SBox output

Given a mask $m_o$ then a _SBox with output mask_ is a modified AES-SBOX $\mathrm{SBOX'}$ with the property:

$$ \mathrm{SBOX'}(\mathrm{pt}_i \oplus k_i) = \mathrm{SBOX}(\mathrm{pt}_i \oplus k_i) \oplus m_o $$

Where $k_i$ is the $i$-th key byte and $\mathrm{pt}_i$ the $i$-th input byte.


In [ ]:
data = capture(
    platform="cwlitexmega",
    number_of_traces=10000,
    number_of_samples=500,
    fromfile=os.path.abspath("sbox_lookup_masking.c"),
    inputfunction=lambda _: [random.randint(0, 255) for _ in range(16)] + [random.randint(0, 255)],
)

cpa(data, sbox_output_selection_function)